# Решение 2 задания

### Считывание данных

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
print(os.listdir())

['.ipynb_checkpoints', 'main.ipynb', 'main2.ipynb', 'Тестовое задание на технического менеджера-стажера.docx', 'файл 1.txt', 'файл 2.csv']


In [4]:
df = pd.read_csv('файл 2.csv', delimiter = "\t")
df.columns = [i.lower() for i in df.columns]
print(df.shape)
df.head()

(250000, 5)


,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0


In [5]:
print(df.dtypes)

login    object
uid       int64
docid     int64
jud       int64
cjud      int64
dtype: object


In [6]:
# Удаляем дублирующиеся строки
df = df.drop_duplicates()
print(df.shape)
df.head()

(249999, 5)


,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0


In [7]:
# Проверка наличия пропущенных значений
df.isna().sum()

login    0
uid      0
docid    0
jud      0
cjud     0
dtype: int64

### Генерация метрик для итогового датасета

#### Итоговый датасет будет состоять из 4 метрик:
<ol>
<li>uid - id асессора (индекс датасета)</li>
<li>docid_num - общее количество оцениваемых документов</li>
<li>correct_num - общее количество правильных оценок асессора</li>
<li>efficiency - эффективность работы асессоров в процентном соотношении</li>
</ol>  

#### docid_num - общее количество оцениваемых документов

In [8]:
result = df.groupby('uid').docid.count().to_frame()
print(result.shape)
result.head()

(600, 1)


,docid
uid,
0,401
1,412
2,379
3,426
4,418


#### correct_num - общее количество правильных оценок асессора

In [9]:
# Отбираем все правильные решения асессоров
correct = df[df.jud == df.cjud]
print(correct.shape)
correct.head()

(210321, 5)


,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0


In [10]:
# Высчитываем количество правильных решений по каждому асессору
correct_num = correct.groupby('uid').docid.count()
correct_num.name = 'correct_num'
print(correct_num.shape)
correct_num.head()

(600,)


uid
0    336
1    330
2    303
3    196
4    346
Name: correct_num, dtype: int64

In [11]:
# Добавляем correct_num в итоговый датасет
result = pd.merge(result, correct_num, on='uid', how='inner')
print(result.shape)
result.head()

(600, 2)


,docid,correct_num
uid,,
0,401,336
1,412,330
2,379,303
3,426,196
4,418,346


#### efficiency - эффективность работы асессоров в процентном соотношении

In [12]:
# Вычисляем эффективность работы асессоров в процентном соотношении и сортируем по убыванию
efficiency = (result.correct_num * 100) / result.docid
efficiency.name = 'efficiency'
result = pd.merge(result, efficiency, on='uid', how='inner')
result = result.sort_values(by='correct_num',ascending=False)
print(result.shape)
result.head()

(600, 3)


,docid,correct_num,efficiency
uid,,,
191,484,446,92.148760
161,480,432,90.000000
368,467,419,89.721627
212,455,414,90.989011
207,453,411,90.728477
